In [16]:
#import the modules
import psycopg2 as pg
import time
from io import StringIO
import pandas as pd
import os
from datetime import datetime
from datetime import date
import numpy as np
from tqdm.notebook import tqdm
import pyspark
import calendar
from pyspark.sql import SparkSession
from pyspark.sql import Row
from datetime import timedelta
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import date_format
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import norm
import re
########################################################## CONVERTING 1-min to DIFFERENT TIMEFRAMES - UNDERLYING ######################################

def EOD_underlying(ddf,index):
    print("CONVERTING TO EOD")
    final_df = ddf.copy()
    final_df['Date'] = pd.to_datetime(final_df['Date'], format='mixed',dayfirst=True)
    final_df = final_df[(final_df['Time']>=time1) & (final_df['Time']<=time2)]
    final_df.reset_index(drop=True,inplace=True)
    final_df = final_df.sort_values(by=['Date'])
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed',dayfirst=True)

    final_df = final_df.set_index("Datetime")

    ddf = final_df.groupby(['Date', pd.Grouper(freq='B')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum'})
    ddf.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume']
    ddf = ddf.reset_index()
    ddf['Ticker'] = f"{index}".upper()+'.EQ-NSE'

    ddf = ddf.sort_values(by=['Datetime'])
    ddf.reset_index(drop=True,inplace=True)
    ddf = ddf.rename(columns={'Adj_Open':'EQ_Open','Adj_High':'EQ_High','Adj_Low':'EQ_Low','Adj_Close':'EQ_Close','Adj_Volume':'EQ_Volume'})
    ddf = ddf[['Ticker','Date','EQ_Open','EQ_High','EQ_Low','EQ_Close','EQ_Volume']]
    ddf.to_csv(fr"C:\\users\\{admin_path}\\desktop\\{index}_EqData\\{index}_EOD.csv",index=False)
    
def fifteen_underlying(ddf,index):
    print("CONVERTING TO 15min")
    final_df = ddf.copy()
    final_df['Date'] = pd.to_datetime(final_df['Date'], format='mixed',dayfirst=True)
    final_df = final_df[(final_df['Time']>=time1) & (final_df['Time']<=time2)]
    final_df.reset_index(drop=True,inplace=True)
    final_df = final_df.sort_values(by=['Date'])
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed',dayfirst=True)

    final_df = final_df.set_index("Datetime")

    ddf = final_df.groupby(['Date', pd.Grouper(freq='15min')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum'})
    ddf.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume']
    ddf = ddf.reset_index()
    ddf['Ticker'] = f"{index}".upper()+'.EQ-NSE'

    ddf = ddf.sort_values(by=['Datetime'])
    ddf['Time'] = pd.to_datetime(ddf['Datetime']).dt.time
    ddf.reset_index(drop=True,inplace=True)
    ddf = ddf.rename(columns={'Adj_Open':'EQ_Open','Adj_High':'EQ_High','Adj_Low':'EQ_Low','Adj_Close':'EQ_Close','Adj_Volume':'EQ_Volume'})
    ddf = ddf[['Ticker','Date','Time','EQ_Open','EQ_High','EQ_Low','EQ_Close','EQ_Volume']]
    ddf.to_csv(fr"C:\\users\\{admin_path}\\desktop\\{index}_EqData\\{index}_15min.csv",index=False)

def five_underlying(ddf,index):
    print("CONVERTING TO 5min")
    final_df = ddf.copy()
    final_df['Date'] = pd.to_datetime(final_df['Date'], format='mixed',dayfirst=True)
    final_df = final_df[(final_df['Time']>=time1) & (final_df['Time']<=time2)]
    final_df.reset_index(drop=True,inplace=True)
    final_df = final_df.sort_values(by=['Date'])
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed',dayfirst=True)

    final_df = final_df.set_index("Datetime")

    ddf = final_df.groupby(['Date', pd.Grouper(freq='5min')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum'})
    ddf.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume']
    ddf = ddf.reset_index()
    ddf['Ticker'] = f"{index}".upper()+'.EQ-NSE'

    ddf = ddf.sort_values(by=['Datetime'])
    ddf['Time'] = pd.to_datetime(ddf['Datetime']).dt.time
    ddf.reset_index(drop=True,inplace=True)
    ddf = ddf.rename(columns={'Adj_Open':'EQ_Open','Adj_High':'EQ_High','Adj_Low':'EQ_Low','Adj_Close':'EQ_Close','Adj_Volume':'EQ_Volume'})
    ddf = ddf[['Ticker','Date','Time','EQ_Open','EQ_High','EQ_Low','EQ_Close','EQ_Volume']]
    ddf.to_csv(fr"C:\\users\\{admin_path}\\desktop\\{index}_EqData\\{index}_5min.csv",index=False)

def one_underlying(ddf,index):
    final_df = ddf.copy()
    final_df['Date'] = pd.to_datetime(final_df['Date'], format= 'mixed', dayfirst=True)
    final_df = final_df[(final_df['Time']>=time1) & (final_df['Time']<=time2)]
    final_df = final_df.sort_values(by=['Date','Time'])
    final_df.reset_index(drop=True,inplace=True)
    final_df = final_df[['Ticker','Date','Time','Open','High','Low','Close','Volume']]
    final_df.to_csv(fr"C:\\Users\\admin\\desktop\\{index}_EqData\\{index}_1min.csv",index=False)
    
###################################################### CONVERTING 1-min to DIFFERENT TIMEFRAMES - OPTIONS DATA ##########################################

def EOD(ddf,index,schema,hyphen_index):
    print("CONVERTING TO EOD")
    ddf = ddf.rename(columns={'ticker' : 'Ticker',
                            'date' : 'Date',
                            'time' : 'Time',
                            'open' : 'Open',
                            'high' : 'High', 
                            'low' : 'Low',
                            'close' : 'Close',
                            'volume' : 'Volume', 
                            'Open Int' : 'Open Interest'})
    ddf['Date'] = pd.to_datetime(ddf['Date'], dayfirst=True)
    ddf = ddf.sort_values(by=['Date'])
    
    symbol = index.upper()
    j='-' + schema[hyphen_index:]
    schema_find = schema[:hyphen_index].upper()
    
    final_df = ddf.copy()
    final_df['Final_strike'] = final_df['Ticker'].str.replace(j, '')
    final_df['Final_strike'] = final_df['Final_strike'].str.replace(f'{index.upper()}'+schema_find, '').str.replace(f'{index.upper()}','').str.replace('CE', '').str.replace('PE', '')
    final_df['Final_strike'] = final_df['Final_strike'].astype(float)
    final_df['Option_Type'] = final_df['Ticker'].str[-2:]
    
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume',
                                        'Open Interest' : 'Adj_OI',        
                                        'Option_type' : 'Option_Type'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format = 'mixed',dayfirst=True)

    final_df = final_df.set_index("Datetime")
    final_df['Adj_OI_1'] = final_df['Adj_OI']

    df_eod = final_df.groupby(['Final_strike', 'Option_Type', pd.Grouper(freq='B')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum',
                                                          'Adj_OI' : 'first',
                                                          'Adj_OI_1' : 'last'})
    df_eod.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume', 'First_OI', 'Last_OI']
    df_eod = df_eod.reset_index()
    df_eod['rem'] = df_eod['Final_strike']%df_eod['Final_strike'].astype(int)
    df_eod.loc[df_eod['rem'] == 0, 'Ticker'] = symbol + schema_find + j +  df_eod['Final_strike'].astype(int).astype(str) + df_eod['Option_Type']
    df_eod.loc[df_eod['rem'] != 0, 'Ticker'] = symbol + schema_find + j + df_eod['Final_strike'].round(2).astype(str) + df_eod['Option_Type']

    df_eod = df_eod.sort_values(by=['Datetime', 'Final_strike'])
    df_eod = df_eod.rename(columns={'Datetime' : 'Date'})
    df_eod['Date'] = pd.to_datetime(df_eod['Date'],dayfirst=True)
    ## CHECKING IF NULL VALUES
    df_eod['New_OI'] = df_eod['Last_OI']
    df_eod = df_eod.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume','New_OI':'Open_Interest'})
    df_eod = df_eod.drop(['First_OI','Last_OI','Option_Type','Final_strike','rem'],axis=1)
    df_eod = df_eod[['Ticker','Date','Open','High','Low','Close','Volume','Open_Interest']]
    df_eod.reset_index(drop=True,inplace=True)
    df_eod.to_csv(fr"C:\\users\\{admin_path}\\desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_EOD.csv", mode='a', header = not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_EOD.csv"), index=False)

def fifteen_min(ddf,index,schema,hyphen_index):
    print("CONVERTING TO 15Min")
    ddf = ddf.rename(columns={'ticker' : 'Ticker',
                            'date' : 'Date',
                            'time' : 'Time',
                            'open' : 'Open',
                            'high' : 'High', 
                            'low' : 'Low',
                            'close' : 'Close',
                            'volume' : 'Volume', 
                            'Open Int' : 'Open Interest'})
    ddf['Date'] = pd.to_datetime(ddf['Date'], dayfirst=True)
    ddf = ddf.sort_values(by=['Date'])
    
    symbol = index.upper()
    j='-' + schema[hyphen_index:]
    schema_find = schema[:hyphen_index].upper()

    final_df = ddf.copy()
    final_df['Final_strike'] = final_df['Ticker'].str.replace(j, '')
    final_df['Final_strike'] = final_df['Final_strike'].str.replace(f'{index.upper()}'+schema_find, '').str.replace(f'{index.upper()}','').str.replace('CE', '').str.replace('PE', '')
    final_df['Final_strike'] = final_df['Final_strike'].astype(float)
    final_df['Option_Type'] = final_df['Ticker'].str[-2:]
    
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume',
                                        'Open Interest' : 'Adj_OI',        
                                        'Option_type' : 'Option_Type'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],format='mixed',dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed',dayfirst=True)

    final_df = final_df.set_index("Datetime")
    final_df['Adj_OI_1'] = final_df['Adj_OI']

    df_eod = final_df.groupby(['Final_strike', 'Option_Type', pd.Grouper(freq='15min')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum',
                                                          'Adj_OI' : 'first',
                                                          'Adj_OI_1' : 'last'})
    df_eod.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume', 'First_OI', 'Last_OI']
    df_eod = df_eod.reset_index()
    df_eod['rem'] = df_eod['Final_strike']%df_eod['Final_strike'].astype(int)
    df_eod.loc[df_eod['rem'] == 0, 'Ticker'] = symbol + schema_find + j +  df_eod['Final_strike'].astype(int).astype(str) + df_eod['Option_Type']
    df_eod.loc[df_eod['rem'] != 0, 'Ticker'] = symbol + schema_find + j + df_eod['Final_strike'].round(2).astype(str) + df_eod['Option_Type'] 

    df_eod = df_eod.sort_values(by=['Datetime', 'Final_strike'])
    df_eod = df_eod.rename(columns={'Datetime' : 'Date'})
    df_eod['Time'] = pd.to_datetime(df_eod['Date']).dt.time
    df_eod['Date'] = pd.to_datetime(df_eod['Date'],dayfirst=True).dt.date
    ## CHECKING IF NULL VALUES
    df_eod['New_OI'] = df_eod['Last_OI']
    df_eod = df_eod.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume','New_OI':'Open_Interest'})
    df_eod = df_eod.drop(['First_OI','Last_OI','Option_Type','Final_strike','rem'],axis=1)
    df_eod = df_eod[['Ticker','Date','Time','Open','High','Low','Close','Volume','Open_Interest']]
    df_eod.reset_index(drop=True,inplace=True)
    df_eod = df_eod.drop_duplicates()
    df_eod.to_csv(fr"C:\\users\\{admin_path}\\desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_15min.csv", mode='a', header = not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_15min.csv"), index=False)
    
def five_min(ddf,index,schema,hyphen_index):
    print("CONVERTING TO 5Min")
    ddf = ddf.rename(columns={'ticker' : 'Ticker',
                            'date' : 'Date',
                            'time' : 'Time',
                            'open' : 'Open',
                            'high' : 'High', 
                            'low' : 'Low',
                            'close' : 'Close',
                            'volume' : 'Volume', 
                            'Open Int' : 'Open Interest'})
    ddf['Date'] = pd.to_datetime(ddf['Date'], dayfirst=True)
    ddf = ddf.sort_values(by=['Date'])
    
    symbol = index.upper()
    j='-' + schema[hyphen_index:]
    schema_find = schema[:hyphen_index].upper()

    final_df = ddf.copy()
    final_df['Final_strike'] = final_df['Ticker'].str.replace(j, '')
    final_df['Final_strike'] = final_df['Final_strike'].str.replace(f'{index.upper()}'+schema_find, '').str.replace(f'{index.upper()}','').str.replace('CE', '').str.replace('PE', '')
    final_df['Final_strike'] = final_df['Final_strike'].astype(float)
    final_df['Option_Type'] = final_df['Ticker'].str[-2:]
    
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume',
                                        'Open Interest' : 'Adj_OI',        
                                        'Option_type' : 'Option_Type'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed',dayfirst=True)

    final_df = final_df.set_index("Datetime")
    final_df['Adj_OI_1'] = final_df['Adj_OI']

    df_eod = final_df.groupby(['Final_strike', 'Option_Type', pd.Grouper(freq='5min')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum',
                                                          'Adj_OI' : 'first',
                                                          'Adj_OI_1' : 'last'})
    df_eod.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume', 'First_OI', 'Last_OI']
    df_eod = df_eod.reset_index()
    df_eod['rem'] = df_eod['Final_strike']%df_eod['Final_strike'].astype(int)
    df_eod.loc[df_eod['rem'] == 0, 'Ticker'] = symbol + schema_find + j +  df_eod['Final_strike'].astype(int).astype(str) + df_eod['Option_Type']
    df_eod.loc[df_eod['rem'] != 0, 'Ticker'] = symbol + schema_find + j + df_eod['Final_strike'].round(2).astype(str) + df_eod['Option_Type'] 

    df_eod = df_eod.sort_values(by=['Datetime', 'Final_strike'])
    df_eod = df_eod.rename(columns={'Datetime' : 'Date'})
    df_eod['Time'] = pd.to_datetime(df_eod['Date']).dt.time
    df_eod['Date'] = pd.to_datetime(df_eod['Date'],dayfirst=True).dt.date
    
    ## CHECKING IF NULL VALUES
    df_eod['New_OI'] = df_eod['Last_OI']
    df_eod = df_eod.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume','New_OI':'Open_Interest'})
    df_eod = df_eod.drop(['First_OI','Last_OI','Option_Type','Final_strike','rem'],axis=1)
    df_eod = df_eod[['Ticker','Date','Time','Open','High','Low','Close','Volume','Open_Interest']]
    df_eod.reset_index(drop=True,inplace=True)
    df_eod.to_csv(fr"C:\\users\\{admin_path}\\desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_5min.csv", mode='a', header = not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_5min.csv"), index=False)

def one_min(ddf,index,schema,hyphen_index):
    ddf = ddf.rename(columns={'ticker' : 'Ticker',
                            'date' : 'Date',
                            'time' : 'Time',
                            'open' : 'Open',
                            'high' : 'High', 
                            'low' : 'Low',
                            'close' : 'Close',
                            'volume' : 'Volume', 
                            'Open Int' : 'Open_Interest'})
    ddf['Date'] = pd.to_datetime(ddf['Date'], dayfirst=True)
    ddf = ddf.sort_values(by=['Date'])
    ddf = ddf[(ddf['Time']>=time1) & ((ddf['Time']<=time2))]
    ddf = ddf.sort_values(by=['Date','Time'])
    ddf.reset_index(drop=True,inplace=True)
    ddf = ddf[['Ticker','Date','Time','Open','High','Low','Close','Volume','Open_Interest']]
    ddf.to_csv(fr"C:\\users\\{admin_path}\\desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_1min.csv", mode='a', header = not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_1min.csv"), index=False)
    
########################################################## OPTIONS DATA FUNCTION ###############################################################

def option_data(index,date1,date2,conversion,schema):
    hyphen_index = schema.find("I")

    st=time.time()
    ## CREATING A DIRECTORY OF THE REQUIRED INDEX AND SCHEMA
    if not os.path.exists(rf"C:\\users\\admin\\desktop\\{index}_{schema}_Data\\"):
        os.makedirs(rf"C:\users\admin\desktop\\{index}_{schema}_Data\\")

    date1 = datetime.strptime(date1, "%Y-%m-%d").date()
    date2 = datetime.strptime(date2, "%Y-%m-%d").date()
    year1 = date1.year
    year2 = date2.year
    print("\nGENERATING OPTIONS DATA")
    for i in range(int(year1),int(year2)+1):
        ddate1 = '-01-01'
        ddate2 = '-12-31'
        year_start = str(str(i)+ddate1)
        year_end = str(str(i)+ddate2)
        year_start = datetime.strptime(year_start, "%Y-%m-%d").date()
        year_end = datetime.strptime(year_end, "%Y-%m-%d").date()
        if date1 > year_start :
            year_start = date1
        else:
            year_start = year_start
        if year_end > date2 :
            year_end = date2
        else:
            year_end = year_end

        ddf = pd.DataFrame()
        engine = pg.connect(f"dbname='{index}db' user='postgres' host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com' port='5432' password='swancap123'")
        print("Generating data from ", year_start , " to " , year_end)
        ddf = pd.read_sql(f'select * from "{index}{schema}".select_datewise(\'{year_start}\',\'{year_end}\')',con=engine)
        ddf = ddf.sort_values(by=["date",'time'])
        ddf = ddf.drop_duplicates()
        ddf.reset_index(drop=True,inplace=True)
        # with open(f"C:\\Users\\Admin\\Desktop\\{index}_{schema}_Data\\{index}_{schema}_"+str(i)+".csv", "w") as file:
        #     cursor.copy_expert(sql, file)
        if conversion == 'E':
            EOD(ddf,index,schema,hyphen_index)
    
        elif conversion == '15':
            fifteen_min(ddf,index,schema,hyphen_index)
    
        elif conversion == '5':
            five_min(ddf,index,schema,hyphen_index)
    
        elif conversion == '1':
            one_min(ddf,index,schema,hyphen_index)

        elif conversion == 'a' or conversion == 'A':
            EOD(ddf,index,schema,hyphen_index)
            fifteen_min(ddf,index,schema,hyphen_index)
            five_min(ddf,index,schema,hyphen_index)
            one_min(ddf,index,schema,hyphen_index)

    et=time.time()
    elapsed_time=et-st;
    print("OPTIONS DATA GENERATED!")
    print("elapsed_time:",elapsed_time)

########################################################## UNDERLYING DATA FUNCTION ###############################################################

def underlying_data(index,date1,date2,conversion):

    if not os.path.exists(rf"C:\\users\\{admin_path}\\desktop\\{index}_EqData\\"):
        os.makedirs(rf"C:\\users\\{admin_path}\\desktop\\{index}_EqData\\")
    
    start_date = datetime.strptime(date1,"%Y-%m-%d")
    end_date = datetime.strptime(date2,"%Y-%m-%d")
    start_date=start_date.date()
    end_date = end_date.date()
    print("\nGENERATING UNDERLYING DATA")
    st = time.time()
    engine = pg.connect("dbname='IndexEQ' user='postgres' host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com' port='5432' password='swancap123'")
    ddf = pd.read_sql(f'select * from "{index}"."AllData" where "Date" between \'{date1}\' and \'{date2}\'', con=engine)
    ddf['Ticker'] = f'{index.upper()}' + '.EQ-NSE'
    ddf = ddf.sort_values(by=["Date",'Time'])

    if conversion == 'E':
        EOD_underlying(ddf,index)
    elif conversion == '15':
        fifteen_underlying(ddf,index)
    elif conversion == '5':
        five_underlying(ddf,index)
    elif conversion == '1':
        one_underlying(ddf,index)
    elif conversion == 'a' or conversion == 'A':
        EOD_underlying(ddf,index)
        fifteen_underlying(ddf,index)
        five_underlying(ddf,index)
        one_underlying(ddf,index)

    et=time.time()
    elapsed_time=et-st;
    print("UNDERLYING DATA GENERATED!")
    print("elapsed_time:",elapsed_time)
    engine.close()

######################################################## MAIN CODE STARTS FROM HERE ######################################################################
admin_path = 'admin'
time1 = datetime.strptime('09:15:00','%H:%M:%S').time()
time2 = datetime.strptime('15:30:00','%H:%M:%S').time()

data = input("Enter O for Options data, U for Underlying data, B for Both the data, E for Exiting ")

################################################ TAKING INPUTS FOR INDEX, DATE RANGE AND TIMEFRAME #############################################

if data == 'o' or data == 'O' or data == 'b' or data == 'B':
    index = input("Enter the index you want in the format below - \nBankNifty\nNifty\nFinNifty ")
    schema = input("Enter schema (MonthlyI, MonthlyII , WeeklyI , QuarterlyI and so on) - ")
    date1 = input("Enter start date as YYYY-MM-DD ")
    date2 = input("Enter end date as YYYY-MM-DD ")
    conversion = input("Enter 1 for 1 minute, 5 for 5 minutes, 15 for 15 minutes, E for EOD, A for All timeframes\n")
    greeks_input = input('Enter Y if you want Greeks or N ')

elif data == 'u' or data == 'U':
    index = input("Enter the index you want in the format below - \nBankNifty\nNifty\nFinNifty\nIndiaVix ")
    date1 = input("Enter start date as YYYY-MM-DD ")
    date2 = input("Enter end date as YYYY-MM-DD ")
    conversion = input("Enter 1 for 1 minute, 5 for 5 minutes, 15 for 15 minutes, E for EOD, A for All timeframes\n")
    greeks_input = 'N'

elif data == 'e' or data == 'E':
    print("Exit!")
    greeks_input = 'N'

else:
    print("Wrong option")

start_time = time.time()
    
if data == 'O' or data == 'o':
    option_data(index,date1,date2,conversion,schema)

elif data == 'U' or data == 'u':
    underlying_data(index,date1,date2,conversion)

elif data == 'B' or data == 'b':
    option_data(index,date1,date2,conversion,schema)
    underlying_data(index,date1,date2,conversion)

if ((greeks_input == 'y' or greeks_input == 'Y') & (data != 'e' or data != 'E')):
    
    if conversion == 'E':
        def greeks(Spot, Strike, MTE, OPT, Option_Type):
            if Option_Type == 'CE':
                NormDist = norm.cdf
                Rate = 0.065
                Dividend = 0.015
                Upper = 5
                Lower = 0
                TargetCE = OPT
                if MTE == 0:
                    CallIV = 5
                    CallDelta = 1 if Spot > Strike else 0
                    CallGamma = 0
                    CallTheta = 0
                    CallVega = 0
                else:
                    while ((Upper - Lower) > 0.00001):   
                        dOne = (math.log(Spot/Strike)+((Rate - Dividend + (0.5 * (((Upper + Lower)/2)**2)))*MTE))/(((Upper + Lower)/2)*(MTE**0.5))
                        dTwo = dOne - ((Upper + Lower)/2) * MTE ** 0.5
                        ndOne = NormDist(dOne)
                        ndTwo = NormDist(dTwo)
                        CallPremium =  (math.exp(-Dividend * MTE) * (Spot * ndOne)) - (Strike * math.exp(-Rate * MTE) * ndTwo)
                        if(CallPremium > TargetCE):
                            Upper = (Upper + Lower) / 2
                        else:
                            Lower = (Upper + Lower) / 2
                    CallIV = (Upper + Lower) / 2  
                    ndashOne = 1 / (2 * math.pi) ** 0.5 * (math.exp(-(dOne**2)/2))
                    CallVega = Spot*((MTE**(1/2))*ndashOne*math.exp(-Dividend*MTE))/100
                    CallGamma = ((((1/math.sqrt((2*math.pi)))*math.exp(((-1*(dOne**2))/2)))*math.exp(((-1*MTE)*Dividend)))/((Spot*CallIV)*math.sqrt(MTE)))
                    CallDelta = ndOne * math.exp(-Dividend * MTE)
                    CallTheta = ((((-1*((((Spot*((1/math.sqrt((2*math.pi)))*math.exp(((-1*(dOne**2))/2))))*CallIV)*math.exp(((-1*MTE)*Dividend)))/(2*math.sqrt(MTE))))+((Dividend*Spot)*CallDelta))-(((Rate*Strike)*math.exp(((-1*Rate)*MTE)))*NormDist(dTwo))))/365
                return CallIV, CallDelta, CallGamma, CallTheta, CallVega
                        
            elif Option_Type == 'PE':
                NormDist = norm.cdf
                Rate = 0.065
                Dividend = 0.015
                Upper = 5
                Lower = 0
                TargetPE = OPT
                if MTE==0:
                    PutIV = 5
                    PutDelta = -1 if Spot < Strike else 0
                    PutGamma = 0
                    PutTheta = 0
                    PutVega = 0
                else:
                    while ((Upper - Lower) > 0.00001):
                        dOne = (math.log(Spot/Strike)+((Rate - Dividend + (0.5*(((Upper + Lower)/2)**2)))*MTE))/(((Upper + Lower)/2)*MTE**0.5)
                        dTwo = dOne - ((Upper + Lower)/2) * MTE ** 0.5
                        ndOne = NormDist(dOne)
                        ndTwo = NormDist(dTwo)
                        PutPremium =  math.exp(-Rate * MTE) * Strike * (NormDist(-dTwo)) -math.exp(-Dividend * MTE) *  Spot* (NormDist( -dOne))
                        if(PutPremium > TargetPE):
                            Upper = (Upper + Lower) / 2
                        else:
                            Lower = (Upper + Lower) / 2
                    PutIV = (Upper + Lower) / 2
                    ndashOne = 1 / (2*math.pi) ** 0.5 * (math.exp(-(dOne**2)/2))
                    PutVega = Spot*((MTE**(1/2))*ndashOne*math.exp(-Dividend*MTE))/100
                    PutGamma = ((((1/math.sqrt((2*math.pi)))*math.exp(((-1*(dOne**2))/2)))*math.exp(((-1*MTE)*Dividend)))/((Spot*PutIV)*math.sqrt(MTE)))
                    PutTheta = (((((-1*((((Spot*((1/math.sqrt((2*math.pi)))*math.exp(((-1*dOne**2)/2))))*PutIV)*math.exp(((-1*MTE)*Dividend)))))/(2*math.sqrt(MTE)))-(((Dividend*Spot)*NormDist((-1*dOne)))*math.exp(((-1*MTE)*Dividend))))+(((Rate*Strike)*math.exp(((-1*Rate)*MTE)))*NormDist((-1*dTwo)))))/365
                    PutDelta = (ndOne-1) * math.exp(-Dividend * MTE)
                return PutIV, PutDelta, PutGamma, PutTheta, PutVega
        
        st_g = time.time()
        eq_df = pd.read_csv(rf"C:\\users\\admin\\desktop\\{index}_EqData\\{index}_EOD.csv",parse_dates=['Date'])
        opt_df = pd.read_csv(fr"C:\Users\admin\Desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_EOD.csv",parse_dates=['Date'])
        eq_df = eq_df.rename(columns={' Time' : 'Time','Open' : 'EQ_Open','High' : 'EQ_High','Low' : 'EQ_Low','Close' : 'EQ_Close'})
        eq_df['Date'] = pd.to_datetime(eq_df['Date'],format='mixed',dayfirst=True)
        opt_df = opt_df.rename(columns={'date' : 'Date',
                                'ticker' : 'Ticker',
                                'time' : 'Time',
                                'open' : 'Open',
                                'high' : 'High',
                                'low' : 'Low',
                                'close' : 'Close',
                                'volume' : 'Volume',
                                'Open_Interest' : 'OpenInterest'})
        opt_df = opt_df.rename(columns={'Open' : 'Adj_Open','High' : 'Adj_High','Low' : 'Adj_Low','Close' : 'Adj_Close'})
        
        opt_df['Date'] = pd.to_datetime(opt_df['Date'], format='mixed',dayfirst=True)
        opt_df['Year'] = opt_df['Date'].dt.year
        opt_df = opt_df.drop(['Year'], axis=1)
        opt_df = opt_df.merge(eq_df[['Date','EQ_Open','EQ_High','EQ_Low','EQ_Close']], on=['Date'], how='left')
        
        exp_df['Date'] = pd.to_datetime(exp_df['Date'], format='mixed',dayfirst=True)
        if schema[schema.find('y')+1:] == 'I':
            opt_df = opt_df.merge(exp_df[['Date', 'E1']], on=['Date'], how='left')
            opt_df = opt_df.rename(columns={'E1' : 'Expiry_Date'})
        elif schema[schema.find('y')+1:] == 'II':
            opt_df = opt_df.merge(exp_df[['Date', 'E2']], on=['Date'], how='left')
            opt_df = opt_df.rename(columns={'E2' : 'Expiry_Date'})
        elif schema[schema.find('y')+1:] == 'III':
            opt_df = opt_df.merge(exp_df[['Date', 'E3']], on=['Date'], how='left')
            opt_df = opt_df.rename(columns={'E3' : 'Expiry_Date'})
        
        opt_df = opt_df.rename(columns={'E1' : 'Expiry_Date'})
        
        opt_df['Date'] = pd.to_datetime(opt_df['Date'],dayfirst=True)
        opt_df['Expiry_Date'] = pd.to_datetime(opt_df['Expiry_Date'], dayfirst=True)
        opt_df['YTE'] = (opt_df['Expiry_Date'] - opt_df['Date']).dt.days
        opt_df['YTE/365'] = opt_df['YTE']/365
        
        opt_df['Final_strike'] = opt_df['Ticker'].str.replace(f'{index.upper()}{schema[:schema.find("y")+1].upper()}-{schema[schema.find("y")+1:]}',"").str[:-2].astype(int)    
        # opt_df['Final_strike'] = opt_df['Ticker'].str.replace(f'{stock_name}-I','').str[:-2].astype(float)#.str.replace(f'{schema_name+hyphens[k]}', '').str.replace(f'{index[i].upper()+hyphens[k]}','').str.replace(f'{index[i].upper()+schema_name+hyphens[k]}','').str.replace(f'{index[i].upper()}','')
        opt_df['Option_Type'] = opt_df['Ticker'].str[-2:]
        
        greeks_series = opt_df.apply(lambda x: greeks(x['EQ_Close'], x['Final_strike'], x['YTE/365'], x['Adj_Close'], x['Option_Type']), axis=1)
        #df = df.drop(['Delta'], axis=1)
        try:
            opt_df[['IV', 'Delta', 'Gamma', 'Theta', 'Vega']] = pd.DataFrame(list(greeks_series))
        except:
            print(opt_df)
            print(opt_df)
        opt_df = opt_df.drop(['YTE', 'YTE/365'], axis=1)
        opt_df = opt_df.sort_values(by=['Date', 'Option_Type', 'Final_strike'])
        # df[['IV', 'Delta', 'Gamma', 'Theta', 'Vega']] = pd.DataFrame(list(greeks_series))
        # df = df.sort_values(by=['Date', 'Time', 'Option_Type', 'Final_strike'])
        # df = df[['Date', 'Time', 'Ticker','IV', 'Delta', 'Gamma', 'Theta', 'Vega']]
        opt_df.to_csv(fr'C:\\users\admin\\desktop\\{index}_{schema}_Greeks_EOD.csv',index=False)
        print("\nGreeks Generated!")
        et_g = time.time()
        print("Time taken to generate greeks",et_g-st_g)
        # file paths
        output_path = r"C:\\users\\admin\\desktop\\"
        # select delta values
        delta_list = [0.05, 0.10, 0.15, 0.20, 0.25,
                      0.30, 0.35, 0.40, 0.45, 0.50, 
                      0.55, 0.60, 0.65, 0.70, 0.75, 
                      0.80, 0.85, 0.90, 0.95]
        
        # load main csv
        print('Start time : ', datetime.now())
        
        opt_df.rename(columns={
                           'Adj_Open' : 'OPT_Open',
                           'Adj_High' : 'OPT_High',
                           'Adj_Low' : 'OPT_Low',
                           'Adj_Close' : 'OPT_Close',
                           'Adj_Volume' : 'OPT_Contracts',
                           # 'Volume' : 'OPT_Contracts',
                            #'New_OI' : 'OPT_OI',
                           'Last_OI' : 'OPT_OI',
                           'OpenInterest' : 'OPT_OI',
                           'Final_strike' : 'Strike',
                           'CONTRACTS' : 'OPT_Contracts',
                           'OPEN_INT' : 'OPT_OI'}, inplace=True)
        
        # extract Strike and Option Type from the Ticker if it's not there in the input file already.
        
        # df['Strike'] = df['Ticker'].str.extract('([0-9]+[./]*[0-9]*)').astype(float)
        
        # df['Option_Type'] = df["Ticker"].str[-2:]
        
        # df['Option_Type'] = np.where((df['Option_Type'] == 'CE') | (df['Option_Type'] == 'PE'),
        #                              df['Option_Type'], 'XX')
        # get the symbol from filename
        symbolFilename = index.upper()+schema.upper()
        print('symbolFilename :', symbolFilename)
        
        # remove '-I'/'-II'/'-III' from filename
        symbol = symbolFilename.replace('-III', '')
        symbol = symbol.replace('-VIII', '')
        symbol = symbol.replace('-VII', '')
        symbol = symbol.replace('-VI', '')
        symbol = symbol.replace('-V', '')
        symbol = symbol.replace('-IV', '')
        symbol = symbol.replace('-II', '')
        symbol = symbol.replace('-I', '')
        print('symbol :', symbol)
        
        # substitute any special characters ('-', '&', '_') in Dispersion file names with empty string ('_')
        charactersDroppedSymbol = re.sub('\ |\_|\.|\-|\&|\;|\:', '', symbol)
        df2 = opt_df.copy()
        # calculate difference between 'EQ_Close' and 'Strike' and get the minimum difference for a group
        df2['Difference'] = abs(df2['EQ_Close'] - df2['Strike'].astype(float))
        dfg = df2.groupby(['Date', 'Option_Type'])['Difference']
        df2['Min'] = dfg.transform('min')
        
        # delete output file if it already exists
        try:
            os.remove(output_path + symbolFilename + '.csv')
        except Exception as e:
            #print('e1 : ', e)
            pass
        
        # get 'At The Money' for each group
        dfg = df2.groupby(['Date', 'Option_Type'])
        for name, group in tqdm(dfg):
            # get 'At The Money' for each group
            try:
                atTheMoney = max(group[(group['Difference'] == group['Min'])]['Strike'])
                group['At_The_Money'] = atTheMoney
            
            except Exception as e:
                #print('e3 : ', e)
                group['At_The_Money'] = np.nan
                
            group = group[['Date', 'Ticker', 'OPT_Open', 'OPT_High', 'OPT_Low', 'OPT_Close', 'Volume', 'OPT_OI','EQ_Open',
                       'EQ_High','EQ_Low','EQ_Close','Expiry_Date','Strike','Option_Type', 'IV', 'Delta', 'Theta', 'Gamma', 'Vega',
                        'At_The_Money']]
            
            # loop through different delta values
            for delta in delta_list:
                if delta == 0.50:
                    group.rename(columns={'Min' : f'Delta_{delta*100:.0f}_Diff_Min',
                                          'Difference' : f'Delta_{delta*100:.0f}_Diff'}, 
                                 inplace=True)
                    group[f'Delta_{delta*100:.0f}_Strike'] = group['At_The_Money']
                else:
                    if group['Option_Type'].iloc[0] == 'CE':
                        group[f'Delta_{delta*100:.0f}_Diff'] = abs(group['Delta'] - delta)
                    elif group['Option_Type'].iloc[0] == 'PE':
                        group[f'Delta_{delta*100:.0f}_Diff'] = abs(group['Delta'] - (-delta))
                    elif group['Option_Type'].iloc[0] == 'XX':
                        group[f'Delta_{delta*100:.0f}_Diff'] = np.nan
                    group[f'Delta_{delta*100:.0f}_Diff_Min'] = group[f'Delta_{delta*100:.0f}_Diff'].min()        
        
                    try:
                        deltaStrike = max(group[group[f'Delta_{delta*100:.0f}_Diff'] == group[f'Delta_{delta*100:.0f}_Diff_Min']]['Strike'])
                        group[f'Delta_{delta*100:.0f}_Strike'] = deltaStrike
                    except Exception as e:
                        #print('e2 : ', e)
                        group[f'Delta_{delta*100:.0f}_Strike'] = np.nan
                    # dropping unnecessary columns
                    group = group.drop([f'Delta_{delta*100:.0f}_Diff_Min', f'Delta_{delta*100:.0f}_Diff'], axis=1)
            # write output to csv
            group.to_csv(output_path + symbolFilename + conversion + '.csv', mode='a', header=not os.path.exists(output_path + symbolFilename + conversion + '.csv'), index=False)

        
    elif conversion != 'E':
        def calculate_greeks_vectorized(spot, strike, time_to_expiry, option_price, option_type):
            norm_dist = norm.cdf
            rate = 0.065
            dividend = 0.015
            is_call_option = np.where(option_type == 'CE', True, False)
            is_put_option = np.where(option_type == 'PE', True, False)
            target_option = np.where(is_call_option, option_price, 0)
            upper = np.where(is_call_option, 5.0, 0.1)
            lower = np.where(is_call_option, 0.0, 0.0)
            while np.any((upper - lower) > 0.00001):
                d_one = (np.log(spot / strike) + ((rate - dividend + (0.5 * (((upper + lower) / 2) ** 2))) * time_to_expiry)) / (((upper + lower) / 2) * (time_to_expiry ** 0.5))
                d_two = d_one - ((upper + lower) / 2) * time_to_expiry ** 0.5
                nd_one = norm_dist(d_one)
                nd_two = norm_dist(d_two)
                call_premium = (np.exp(-dividend * time_to_expiry) * (spot * nd_one)) - (strike * np.exp(-rate * time_to_expiry) * nd_two)
                mask = call_premium > target_option
                upper = np.where(mask, (upper + lower) / 2, upper)
                lower = np.where(mask, lower, (upper + lower) / 2)
            calldelta1=np.where(is_call_option,(nd_one) * np.exp(-dividend * time_to_expiry),0)
            call_delta=np.where((is_call_option)&(time_to_expiry==0)&(spot > strike), 1.0, calldelta1)
            call_iv = np.where(is_call_option, (upper + lower) / 2, 0.0)
            ndash_one = 1 / (2 * np.pi) ** 0.5 * (np.exp(-(d_one ** 2) / 2))
            call_vega = np.where(is_call_option, spot * ((time_to_expiry ** (1 / 2)) * ndash_one * np.exp(-dividend * time_to_expiry)) / 100, 0.0)
            call_gamma = np.where(is_call_option, ((((1 / np.sqrt((2 * np.pi))) * np.exp(((-1 * (d_one ** 2)) / 2))) * np.exp(((-1 * time_to_expiry) * dividend))) / ((spot * call_iv) * np.sqrt(time_to_expiry))), 0.0)
            call_theta =np.where(is_call_option,((((-1 * ((((spot * ((1 / np.sqrt((2 * np.pi))) * np.exp(((-1 * (d_one ** 2)) / 2)))) * call_iv) * np.exp(((-1 * time_to_expiry) * dividend))) / (2 * np.sqrt(time_to_expiry)))) + ((dividend * spot) * call_delta)) - (((rate * strike) * np.exp(((-1 * rate) * time_to_expiry))) * norm_dist(d_two)))) / 365,0)
            target_option1 = np.where(is_put_option, option_price, 0)
            upper1 = np.where(is_put_option, 5.0, 0.1)
            lower1 = np.where(is_put_option, 0.0, 0.0)
            while np.any((upper1 - lower1) > 0.00001):
                d_one1 = (np.log(spot / strike) + ((rate - dividend + (0.5 * (((upper1 + lower1) / 2) ** 2))) * time_to_expiry)) / (((upper1 + lower1) / 2) * (time_to_expiry ** 0.5))
                d_two1 = d_one1 - ((upper1 + lower1) / 2) * time_to_expiry ** 0.5
                nd_one1 = norm_dist(d_one1)
                nd_two1 = norm_dist(d_two1)
                put_premium =  np.exp(-rate * time_to_expiry) * strike * (norm_dist(-d_two1)) -np.exp(-dividend * time_to_expiry) *  spot* (norm_dist( -d_one1))
                mask1 = put_premium > target_option1
                upper1 = np.where(mask1, (upper1 + lower1) / 2, upper1)
                lower1 = np.where(mask1, lower1, (upper1 + lower1) / 2)
            put_delta1 = np.where(is_put_option, (nd_one1 - 1) * np.exp(-dividend * time_to_expiry), 0.0)
            put_delta=np.where((is_put_option)&(time_to_expiry==0)&(spot > strike), 1.0, put_delta1)
            put_iv = np.where(is_put_option, (upper1 + lower1) / 2, 0.0)
            ndash_one1 = 1 / (2 * np.pi) ** 0.5 * (np.exp(-(d_one1 ** 2) / 2))
            put_vega = np.where(is_put_option, spot * ((time_to_expiry ** (1 / 2)) * ndash_one1 * np.exp(-dividend * time_to_expiry)) / 100, 0.0)
            put_gamma = np.where(is_put_option, ((((1 / np.sqrt((2 * np.pi))) * np.exp(((-1 * (d_one1 ** 2)) / 2))) * np.exp(((-1 * time_to_expiry) * dividend))) / ((spot * put_iv) * np.sqrt(time_to_expiry))), 0.0)
            put_theta =  np.where(is_put_option,(((((-1 * ((((spot * ((1 / np.sqrt((2 * np.pi))) * np.exp(((-1 * d_one1 ** 2) / 2)))) * put_iv) * np.exp(((-1 * time_to_expiry) * dividend)))))/ (2 * np.sqrt(time_to_expiry))) - (((dividend * spot) * norm_dist((-1 * d_one1))) * np.exp(((-1 * time_to_expiry) * dividend)))) + (((rate * strike) * np.exp(((-1 * rate) * time_to_expiry))) * norm_dist((-1 * d_two1))))) / 365,0.0)
            iv= np.where(is_call_option,call_iv,put_iv)
            delta= np.where(is_call_option,call_delta,put_delta)
            gamma= np.where(is_call_option,call_gamma,put_gamma)
            theta= np.where(is_call_option,call_theta,put_theta)
            vega=np.where(is_call_option,call_vega,put_vega)
            return iv, delta, gamma, theta, vega
        
        time1 = datetime.strptime('15:29:59', '%H:%M:%S').time()
        st_g = time.time()
        ################## EQUITY DATA
        eq_df = pd.read_csv(rf"C:\\users\\admin\\desktop\\{index}_EqData\\{index}_{conversion}min.csv")
        eq_df = eq_df.rename(columns={' Time' : 'Time','Open' : 'EQ_Open','High' : 'EQ_High','Low' : 'EQ_Low','Close' : 'EQ_Close'})
        eq_df = eq_df[['Date', 'Time', 'EQ_Open', 'EQ_High', 'EQ_Low', 'EQ_Close']]
        eq_df['Date'] = pd.to_datetime(eq_df['Date'], format = 'mixed',dayfirst=True)
        eq_df['Time'] = pd.to_datetime(eq_df['Time']).dt.time
        
        ################## OPTIONS DATA
        opt_df = pd.read_csv(rf"C:\\users\\admin\\desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_{conversion}min.csv")
        opt_df = opt_df.rename(columns={'date' : 'Date','ticker' : 'Ticker','time' : 'Time','open' : 'Open','high' : 'High','low' : 'Low','close' : 'Close','volume' : 'Volume','Open Int' : 'OpenInterest'})
        opt_df = opt_df.rename(columns={'Open' : 'Adj_Open','High' : 'Adj_High','Low' : 'Adj_Low','Close' : 'Adj_Close'})
        opt_df['Date'] = pd.to_datetime(opt_df['Date'], dayfirst=True , format = '%Y-%m-%d')
        opt_df['Year'] = opt_df['Date'].dt.year
        opt_df = opt_df.drop(['Year'], axis=1)
        opt_df['Time'] = pd.to_datetime(opt_df['Time']).dt.time
        opt_df = opt_df[opt_df['Time'] <= time1]
        opt_df = opt_df.merge(eq_df, on=['Date', 'Time'], how='left')
        ################# EXPIRY SHEET
        if index != 'FinNifty':
            engine = pg.connect("dbname='ExpiryDates' user='postgres' host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com' port='5432' password='swancap123'")
            exp_df = pd.read_sql(f'select * from "public"."{schema[:schema.find("y")+1]}Expiry"', con=engine)
            engine.close()
        elif index == 'FinNifty':
            engine = pg.connect("dbname='ExpiryDates' user='postgres' host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com' port='5432' password='swancap123'")
            exp_df = pd.read_sql(f'select * from "public"."{index}{schema[:schema.find("y")+1]}Expiry"', con=engine)
            engine.close()
        exp_df['Date'] = pd.to_datetime(exp_df['Date'], format='mixed',dayfirst=True)
        if schema[schema.find('y')+1:] == 'I':
            opt_df = opt_df.merge(exp_df[['Date', 'E1']], on=['Date'], how='left')
            opt_df = opt_df.rename(columns={'E1' : 'Expiry_Date'})
        elif schema[schema.find('y')+1:] == 'II':
            opt_df = opt_df.merge(exp_df[['Date', 'E2']], on=['Date'], how='left')
            opt_df = opt_df.rename(columns={'E2' : 'Expiry_Date'})
        elif schema[schema.find('y')+1:] == 'III':
            opt_df = opt_df.merge(exp_df[['Date', 'E3']], on=['Date'], how='left')
            opt_df = opt_df.rename(columns={'E3' : 'Expiry_Date'})
        opt_df['Date'] = opt_df['Date'].astype(str)
        opt_df['Time'] = opt_df['Time'].astype(str)
        opt_df['Expiry_Date'] = opt_df['Expiry_Date'].astype(str)
        opt_df['Datetime'] = pd.to_datetime(opt_df['Date'] + ' ' + opt_df['Time'], dayfirst=True,format = '%Y-%m-%d %H:%M:%S')
        opt_df['Expiry_Datetime'] = pd.to_datetime(opt_df['Expiry_Date'] + ' ' + '15:30:00', dayfirst=True)
        opt_df['MTE'] = opt_df['Expiry_Datetime'] - opt_df['Datetime']
        opt_df['MTE'] = opt_df['MTE'].dt.total_seconds()/60
        opt_df['MTE'] = opt_df['MTE']/(365 * 24 * 60)
        opt_df['Final_strike'] = opt_df['Ticker'].str.replace(f'{index.upper()}{schema[:schema.find("y")+1].upper()}-{schema[schema.find("y")+1:]}',"").str[:-2].astype(int)
        opt_df['Option_Type'] = opt_df['Ticker'].str[-2:]
        
        greeks=calculate_greeks_vectorized(opt_df['EQ_Close'], opt_df['Final_strike'], opt_df['MTE'], opt_df['Adj_Close'], opt_df['Option_Type'])
        opt_df['IV'], opt_df['Delta'], opt_df['Gamma'], opt_df['Theta'], opt_df['Vega'] = greeks[0], greeks[1], greeks[2], greeks[3], greeks[4]
        opt_df = opt_df.sort_values(by=['Date', 'Time', 'Option_Type', 'Final_strike'])
        opt_df.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close'},inplace=True)
        opt_df = opt_df[['Ticker','Date', 'Time', 'Open','High','Low','Close','Volume','Open_Interest','EQ_Open','EQ_High','EQ_Low','EQ_Close','Expiry_Date','IV', 'Delta', 'Gamma', 'Theta', 'Vega',"Option_Type",'Final_strike']]
        opt_df=opt_df.reset_index(drop=True)
        opt_df = opt_df.drop_duplicates()
        opt_df.to_csv(rf'C:\\users\\admin\\desktop\\{index}_{schema}_Greeks_{conversion}min.csv',index=False)
        print("\nGreeks Generated!")
        et_g = time.time()
        print("Time taken to generate greeks",et_g-st_g)
        
        opt_df = opt_df.sort_values(by=['Date', 'Time', 'Option_Type', 'Final_strike']).reset_index(drop=True)
        output_path = r"C:\\users\\admin\\desktop\\"
        delta_list = [0.05, 0.10, 0.15, 0.20, 0.25,
                      0.30, 0.35, 0.40, 0.45, 0.50, 
                      0.55, 0.60, 0.65, 0.70, 0.75,
                      0.80, 0.85, 0.90, 0.95]
        # delta_list = [0.50]
        
        print('Start time : ', datetime.now())
        # change date column to datetime format
        
        opt_df['Old_Delta'] = opt_df['Delta'].copy()
        if opt_df['Delta'].dtype == object:
            print('Delta column is in string format.')
            opt_df['Delta'] = opt_df['Delta'].str.replace('\(|\)', '')
            opt_df['Delta'] = opt_df['Delta'].str.replace('\+0j', '')
            opt_df['Delta'] = opt_df['Delta'].str.replace('0j', '0')
            opt_df['Delta'] = opt_df['Delta'].astype(float)
        
        opt_df.rename(columns={
                           'Open' : 'OPT_Open',
                           'High' : 'OPT_High',
                           'Low' : 'OPT_Low',
                           'Close' : 'OPT_Close',
                           # 'Volume' : 'OPT_Contracts',
                           'Open_Interest' : 'OPT_OI',
                           'Final_strike' : 'Strike'
                                                }, inplace=True)
        # extract Strike and Option Type from the Ticker if it's not there in the input file already.
        
        #     df['Strike'] = df['Ticker'].str.extract('([0-9]+[./]*[0-9]*)').astype(float)
        
        #     df['Option_Type'] = df["Ticker"].str.split('-').str[0].str[-2:]
        
        #     df['Option_Type'] = np.where((df['Option_Type'] == 'CE') | (df['Option_Type'] == 'PE'),
        #                                  df['Option_Type'], 'XX')
        # get the symbol from filename
        symbolFilename = index.upper()+schema[:schema.find('y')+1].upper()+'-'+schema[schema.find('y')+1:]
        # symbolFilename = index.upper()+schema.upper()
        print('symbolFilename :', symbolFilename)
        
        # remove '-I'/'-II'/'-III' from filename
        symbol = symbolFilename.replace('-III', '')#.replace('III','')
        symbol = symbol.replace('-II', '')#.replace('II', '')
        symbol = symbol.replace('-I', '')#.replace('I', '')
        print('symbol :', symbol)
        
        # substitute any special characters ('-', '&', '_') in Dispersion file names with empty string ('_')
        charactersDroppedSymbol = re.sub('\ |\_|\.|\-|\&|\;|\:', '', symbol)
        
        df2 = opt_df.copy()
        df2['Date'] = pd.to_datetime(df2['Date'],format='mixed',dayfirst=True)
        # calculate difference between 'EQ_Close' and 'Strike' and get the minimum difference for a group
        if schema[schema.find('y')+1:] == 'I':
            df2 = df2.merge(exp_df[['Date', 'E1']], on=['Date'], how='left')
            df2 = df2.rename(columns={'E1' : 'Expiry_Date'})
        elif schema[schema.find('y')+1:] == 'II':
            df2 = df2.merge(exp_df[['Date', 'E2']], on=['Date'], how='left')
            df2 = df2.rename(columns={'E2' : 'Expiry_Date'})
        elif schema[schema.find('y')+1:] == 'III':
            df2 = df2.merge(exp_df[['Date', 'E3']], on=['Date'], how='left')
            df2 = df2.rename(columns={'E3' : 'Expiry_Date'})
        df2['Difference'] = abs(df2['EQ_Close'] - df2['Strike'].astype(float))
        dfg = df2.groupby(['Date', 'Time', 'Option_Type'])['Difference']
        df2['Min'] = dfg.transform('min')
        
        # delete output file if it already exists
        try:
            os.remove(output_path + symbolFilename + '.csv')
        except Exception as e:
            pass
        
        # get 'At The Money' for each group
        dfg = df2.groupby(['Date', 'Time', 'Option_Type'])
        for name, group in tqdm(dfg):
            # get 'At The Money' for each group
            try:
                atTheMoney = max(group[(group['Difference'] == group['Min'])]['Strike'])
                group['At_The_Money'] = atTheMoney
            
            except Exception as e:
                group['At_The_Money'] = np.nan
            
            group = group[['Date', 'Time', 'Ticker', 'OPT_Open', 'OPT_High', 'OPT_Low', 'OPT_Close', 'Volume', 'OPT_OI','EQ_Open',
                       'EQ_High','EQ_Low','EQ_Close', 'Expiry_Date','Strike', 'Option_Type', 'IV', 'Delta', 'Theta', 'Gamma', 'Vega',  
                        'At_The_Money']]  
            # loop through different delta values
            for delta in delta_list:
                
                if delta == 0.50:
                    group.rename(columns={'Min' : f'Delta_{delta*100:.0f}_Diff_Min',
                                          'Difference' : f'Delta_{delta*100:.0f}_Diff'}, 
                                 inplace=True)
                    group[f'Delta_{delta*100:.0f}_Strike'] = group['At_The_Money']
                else:
                    try:
                        if group['Option_Type'].iloc[0] == 'CE':
                            group[f'Delta_{delta*100:.0f}_Diff'] = abs(group['Delta'] - delta)
                        elif group['Option_Type'].iloc[0] == 'PE':
                            group[f'Delta_{delta*100:.0f}_Diff'] = abs(group['Delta'] - (-delta))
                        elif group['Option_Type'].iloc[0] == 'XX':
                            group[f'Delta_{delta*100:.0f}_Diff'] = np.nan
        
                        group[f'Delta_{delta*100:.0f}_Diff_Min'] = group[f'Delta_{delta*100:.0f}_Diff'].min()        
        
                        try:
                            deltaStrike = max(group[group[f'Delta_{delta*100:.0f}_Diff'] == group[f'Delta_{delta*100:.0f}_Diff_Min']]['Strike'])
                            group[f'Delta_{delta*100:.0f}_Strike'] = deltaStrike
                        except Exception as e:
                            group[f'Delta_{delta*100:.0f}_Strike'] = np.nan
                        # dropping unnecessary columns
                        group = group.drop([f'Delta_{delta*100:.0f}_Diff_Min', f'Delta_{delta*100:.0f}_Diff'], axis=1)
                        
                    except Exception as e1:
                        print('e1 : ', e1)
            # write output to csv
            group.to_csv(output_path + symbolFilename + "_" + conversion + 'min.csv', mode='a', header=not os.path.exists(output_path + symbolFilename + "_" + conversion + 'min.csv'), index=False)
        
        del opt_df,eq_df
        
end_time = time.time()
print("\nCOMPLETED.")
print("Total time taken ",end_time-start_time)


Enter O for Options data, U for Underlying data, B for Both the data, E for Exiting  O
Enter the index you want in the format below - 
BankNifty
Nifty
FinNifty  FinNifty
Enter schema (MonthlyI, MonthlyII , WeeklyI , QuarterlyI and so on) -  MonthlyII
Enter start date as YYYY-MM-DD  2021-01-01
Enter end date as YYYY-MM-DD  2023-05-31
Enter 1 for 1 minute, 5 for 5 minutes, 15 for 15 minutes, E for EOD, A for All timeframes
 15
Enter Y if you want Greeks or N  Y



GENERATING OPTIONS DATA
Generating data from  2021-01-01  to  2021-12-31
CONVERTING TO 15Min
Generating data from  2022-01-01  to  2022-12-31
CONVERTING TO 15Min
Generating data from  2023-01-01  to  2023-05-31
CONVERTING TO 15Min
OPTIONS DATA GENERATED!
elapsed_time: 0.8047809600830078

Greeks Generated!
Time taken to generate greeks 0.2790257930755615
Start time :  2023-08-17 11:45:16.014274
symbolFilename : FINNIFTYMONTHLY-II
symbol : FINNIFTYMONTHLY


  0%|          | 0/322 [00:00<?, ?it/s]


COMPLETED.
Total time taken  22.186803579330444
